In [ ]:
import os
import cv2
import numpy as np

def save_edge_image(image_path, output_dir, low_thresh_list, high_thresh_list, prefix="edge"):
    """
    Salva immagini dei bordi con vari parametri di soglia.
    image_path : str → percorso all’immagine originale
    output_dir : str → cartella dove salvare i risultati
    low_thresh_list : list di int
    high_thresh_list : list di int (della stessa lunghezza di low_thresh_list)
    prefix : str → prefisso per i nomi dei file salvati
    """
    # Crea la cartella se non esiste
    os.makedirs(output_dir, exist_ok=True)
    
    # Leggi immagine, converti in scala di grigi
    img_color = cv2.imread(image_path)
    if img_color is None:
        print(f"Errore: immagine non trovata {image_path}")
        return
    img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    
    for low, high in zip(low_thresh_list, high_thresh_list):
        edges = cv2.Canny(img_gray, low, high)
        # Vuoi colorarla in 3 canali per salvarla come RGB
        edges_rgb = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        filename = f"{prefix}_low{low}_high{high}.png"
        out_path = os.path.join(output_dir, filename)
        cv2.imwrite(out_path, edges_rgb)
        print(f"Salvata: {out_path}")


image_path = "./dataset_ruote/train/non_consumate/1000047520.jpg"     # <— Sostituisci con il percorso della tua immagine
output_dir = "edge_tests"

# Lista di soglie da provare
low_thresh_list  = [20, 25, 30, 35]
high_thresh_list = [60, 75, 90, 105]  # normalmente high ~ 2-3× low secondo tutorial :contentReference[oaicite:0]{index=0}

#save_edge_image(image_path, output_dir, low_thresh_list, high_thresh_list)

import os
import cv2
import numpy as np
from PIL import Image, ImageOps
from torchvision import transforms

# ---- 5) Trasformazioni ----
class CenterPadCrop:
    def __init__(self, final_size=224):
        self.final_size = final_size

    def __call__(self, img: Image.Image):
        w, h = img.size
        if h > w:
            new_h = self.final_size
            new_w = int(w * self.final_size / h)
        else:
            new_w = self.final_size
            new_h = int(h * self.final_size / w)
        img = img.resize((new_w, new_h), resample=Image.BILINEAR)

        pad_left = (self.final_size - new_w) // 2
        pad_right = self.final_size - new_w - pad_left
        pad_top = (self.final_size - new_h) // 2
        pad_bottom = self.final_size - new_h - pad_top

        img = transforms.functional.pad(img, padding=(pad_left, pad_top, pad_right, pad_bottom), fill=0)
        return img

transform_rgb = transforms.Compose([
    CenterPadCrop(final_size=224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

edge_transform = transforms.Compose([
    CenterPadCrop(final_size=224),
    transforms.ToTensor(),
])


def save_transformed_images(
    input_dir, 
    output_dir, 
    transform_rgb, 
    edge_transform
):
    """
    Applica le stesse trasformazioni del WheelDataset (CenterPadCrop, ToTensor, ecc.)
    e salva img, edges e combined per TUTTE le immagini della cartella.
    """

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "rgb"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "edges"), exist_ok=True)

    image_list = sorted([
        f for f in os.listdir(input_dir)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    for img_name in image_list:
        img_path = os.path.join(input_dir, img_name)

        # ---- Carica immagine ----
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ---- Converto in PIL ----
        img_pil = Image.fromarray(img)
        img_pil = ImageOps.exif_transpose(img_pil)

        # ---- Applica trasformazioni ----
        img_t = transform_rgb(img_pil)              # (3, 224, 224)
        gray = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2GRAY)
        edges = cv2.Canny(gray, 30, 90)             # array 2D grayscale
        edges_pil = Image.fromarray(edges) 

        # ---- Salvataggio ----
        # Salva RGB
        rgb_out = img_t.permute(1, 2, 0).numpy()
        rgb_out = (rgb_out - rgb_out.min()) / (rgb_out.max() - rgb_out.min()) * 255
        rgb_out = rgb_out.astype(np.uint8)
        Image.fromarray(rgb_out).save(os.path.join(output_dir, "rgb", img_name))

        # Salva edges
        edges_pil.save(
            os.path.join(output_dir, "edges", img_name)
        )

        print(f"✓ Salvato {img_name}")

    print("\n--- COMPLETATO ---")



In [ ]:
input_dir = "./ruote_catalogate_def/1,2/"
output_dir = "./transformed_outputs"

save_transformed_images(
    input_dir=input_dir,
    output_dir=output_dir,
    transform_rgb=transform_rgb,
    edge_transform=edge_transform
)